<a href="https://colab.research.google.com/github/ferjorosa/learn-fastai/blob/main/basic_mnist_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The objective of this notebook is to simply put my notes and examples of the MNIST example of Chapter 4 from the Fast AI book

In [3]:
#hide (Google Colab)
!pip install fastai --upgrade -q
import fastai
print(fastai.__version__)

!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

2.5.3


RuntimeError: ignored

In [ ]:
#hide
from fastai.vision.all import *
from fastbook import *

matplotlib.rc('image', cmap='Greys')